In [1]:
from osgeo import gdal,ogr,osr
import pandas as pd
import geopandas as gpd
from shapely import wkt
from itertools import compress
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import numpy

In [3]:
def ImageBoundry(FilePath):
        #for aspect data
        imgname=FilePath

        from osgeo import gdal,ogr
        ds = gdal.Open(FilePath)
        gt = ds.GetGeoTransform()  # captures origin and pixel size
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(4326)
        

        
        ULC = gdal.ApplyGeoTransform(gt, 0, 0) #Upper Left Corner
        URC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, 0) #Upper Right Corner
        LLC = gdal.ApplyGeoTransform(gt, 0, ds.RasterYSize) #Lower Left Corner
        LRC = gdal.ApplyGeoTransform(gt, ds.RasterXSize, ds.RasterYSize) #Lower Right Corner
                   
        ring = ogr.Geometry(ogr.wkbLinearRing)
        ring.AddPoint_2D(ULC[0], ULC[1])
        ring.AddPoint_2D(URC[0], URC[1])
        ring.AddPoint_2D(LRC[0], LRC[1])
        ring.AddPoint_2D(LLC[0], LLC[1])
        ring.AddPoint_2D(ULC[0], ULC[1]) 
        poly=ogr.Geometry(ogr.wkbPolygon)
        poly.AddGeometry(ring)
        #poly.SpatialReference('EPSG:4326')
        # second item is name of the image
        return poly.ExportToJson() ,imgname

In [2]:
# mosaic_files=glob.glob('../data/Modis_Fire_Date/max_tif_mosaic_4326/*')
# mosaic_files=sorted(mosaic_files)

mosaic_files=glob.glob(r'//akif.internal/public/demo-geo-stack/geoserver/data/im-data-soil/isric-bdticm-m-250m-ll.tif')
mosaic_files=sorted(mosaic_files)

In [4]:
%%time
for img in mosaic_files:
    # define tiles path
    path=r'//akif.internal/public/demo-geo-stack/geoserver/data/im-data-soil/akif_test_mosaic/'
    input_path=img
    # year_info=os.path.basename(img)[-8:-4]
    # try:
    #     # create year folder under target folder (path)
    #     os.mkdir(path+year_info)
    # except:
    #     pass
    
    # # name of tile image
    # output_path=path+year_info+'/modis_MCD64A1_burned_area_'+year_info+'_tile_'
    output_path=path

    ds = gdal.Open(img)
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize
    # define your tile size
    # it could be 256,512 or 1024. it depends on your scope
    tile_size_x = 512
    tile_size_y = 512
    for i in range(0, xsize, tile_size_x):
        if xsize-i < tile_size_x:
            i=xsize-tile_size_x            
        for j in range(0, ysize, tile_size_y):
            if ysize-j < tile_size_y:
                j=ysize-tile_size_y
    
            com_string = "gdal_translate -of GTIFF -srcwin " + str(i)+ ", " + str(j) + ", " + str(tile_size_x) + ", " + str(tile_size_y) + " " + str(input_path)+ " " + str(output_path)  + str(i) + "_" + str(j) + ".tif"
            #print(com_string)
            os.system(com_string)
        
    # with this block, we delete tiles that include just nodata
    tiles=glob.glob(path+year_info+'/*')
    for img in tiles:
        hdf_ds = gdal.Open(img, gdal.GA_ReadOnly)
        array_img=hdf_ds.ReadAsArray()
        np_where_img = numpy.where((array_img<1),-32768,(array_img))
        check=numpy.all(np_where_img==-32768)
        # if np_where_img all value equal nodata value
        # remove the img
        if check:
            os.remove(img)
            continue
    
    array_img=None
    np_where_img=None
    print(path+year_info)

NameError: name 'year_info' is not defined

In [10]:
path=r'//akif.internal/public/demo-geo-stack/geoserver/data/im-data-soil/akif_test_mosaic/'
tiles=glob.glob(path + "*")


# Z:\demo-geo-stack\geoserver\data\im-data-soil\akif_test_mosaic


In [18]:
output_path = r"//akif.internal/public/demo-geo-stack/geoserver/data/im-data-soil/akif_test_mosaic_copy/"

In [15]:
import shutil

In [19]:
shutil.copy(tiles[0],output_path)

'//akif.internal/public/demo-geo-stack/geoserver/data/im-data-soil/akif_test_mosaic_copy/131584_55296.tif'

In [21]:
output_path = r"//akif.internal/public/demo-geo-stack/geoserver/data/im-data-soil/akif_test_mosaic_copy/"

for img in tiles:
    try:
        hdf_ds = gdal.Open(img, gdal.GA_ReadOnly)
        array_img=hdf_ds.ReadAsArray()
        np_where_img = numpy.where((array_img<1),-32768,(array_img))
        check=numpy.all(np_where_img==-32768)
        # if np_where_img all value equal nodata value
        # remove the img
        if not check: #if data has different value
            shutil.copy(img,output_path)
            continue
    except:
        continue
    
array_img=None
np_where_img=None
# print(path+year_info)

In [14]:
!pip install pytest-shutil

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4847 sha256=ab001807ea2a1c48d69140ee1985df9602dde5928ee8d5b551e93992c4b90d9e
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\3f\e3\ec\8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
Successfully built termcolor